<font color=#ff0000> 全国高校计算机骨干教师高级研修班——《强化学习》之第04讲</font>
# <center>&emsp;基于模型的求解方法</center>

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;代码1:基于确定环境动作值函数的策略评估代码实现<br>

In [1]:
# 确定环境机器人寻宝设置
from myGridWorld import GridWorldEnv
import numpy as np
import xlsxwriter

'''环境的设置'''
height = 5                                      # 格子世界高度，对应状态行
width = 5                                       # 格子世界宽度，对应状态列
unitSize = 60                                   # 格子世界在渲染时，每个状态格的边长
defaultReward = 0                               # 到达一般状态的奖赏
actionCount = 4                                 # 动作数
endState = [(4, 0, 1), (1, 4, 3)]               # 终止状态及获得的奖赏
typeState = [(2, 2, -10)]                       # 特殊奖赏状态及获得的奖赏
unreachState = [(2, 2)]                         # 不可到达的状态

edgeReward = -1
'''
动作：上：(-1,0)，下：(1, 0), 左：(0, -1), 右：(0, 1),
    左上：(-1, -1), 左上：(1, -1), 右上：(-1, 1), 右下(1, 1)
'''
actionAllList = [(-1, 0), (1, 0), (0, -1),
              (0, 1), (-1, -1), (1, -1), (-1, 1), (1, 1)]
startState = (1, 1)                             # 格子世界的当前状态
edgeFlag = False                                # 是否给冲出边界的动作，True--有， Flase--没有
env = GridWorldEnv(height,
                   width,
                   unitSize,
                   defaultReward,
                   actionCount,
                   endState,
                   typeState,
                   unreachState,
                   edgeReward,
                   actionAllList,
                   edgeFlag,
                   startState)
env.seed(1)                                     # 提供随机种子

ModuleNotFoundError: No module named 'myGridWorld'

In [ ]:
# 过程----确定环境状态动作值函数评估
from mdpModel import MdpModel, Tools
import numpy as np
class StateActionValue(MdpModel, Tools):
    def calStateActionValue(self, sheet, q, policy, theta, gamma, initPolicyPi, printFlag, outputMidFlag):         # 阈值，设置为一个较小的实值
        self.q = q
        self.printFlag = printFlag
        self.outputMidFlag = outputMidFlag
        tao = 0                                          # 迭代轮次
        self.actionValueOutputExcel(sheet, self.q, tao)
        while True:
            delta = 0                                    # 相对应状态值函数差值的最小值的绝对值，初始化为0
            qBackup = self.q.copy()
            for s in self.stateSpace():
                row, col = s
                actionNumber = 0
                for actionPi in initPolicyPi[row * env.width + col]:
                    if actionPi > 0:
                        act = self.actionSpace()
                        a = act[actionNumber]
                        env.state = s
                        s1, _, _, _ = env.step(actionNumber, self.actionSpace())
                        r = self.rewardFunction(s, a)
                        p = self.transferFunction(s, a, s1)
                        actionNumber2 = 0
                        row1, col1 = s1
                        tempValue = 0.0
                        for actionPi1 in policy[row1 * env.width + col1]:
                            pi = actionPi1
                            tempValue = tempValue + pi * self.q[row1 * env.width + col1][actionNumber2]
                            actionNumber2 += 1
                        self.q[row * env.width + col][actionNumber] = p * (r + gamma * tempValue)
                        delta = max(delta, abs(self.q[row * env.width + col][actionNumber] - qBackup[row * env.width + col][actionNumber]))
                    else:
                        self.q[row * env.width + col][actionNumber] = -99999
                    actionNumber += 1
            tao += 1        
            if self.printFlag:
                print('第%d轮迭代状态动作值'%tao)
                self.printActionValue(self.q)
            if self.outputMidFlag:
                self.actionValueOutputExcel(sheet, self.q, tao)
            if delta < theta:
                break
        return self.q       
            

结果----确定环境状态动作值函数评估结果

In [ ]:
# 结果----确定环境状态动作值函数评估结果
if __name__ == '__main__':
    stateActionValue = StateActionValue(env)
    initPolicyPi  = stateActionValue.stateActionSpacePi()
    q = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
    theta = 0.0001
    gamma = 0.8
    policy = initPolicyPi.copy()
    printFlag = True
    theta = 0.0001
    gamma = 0.8
    printFlag = True                                                  #是否打印评估过程
    outputMidFlag = True                                              #是否输出中间过程到Excel表
    if printFlag:
        print("迭代过程：")
    actionValueExcelFile = xlsxwriter.Workbook(r'.\resultTable\表4.1 初始策略下确定环境机器人寻宝任务的动作值函数评估过程表.xlsx')
    sheet = actionValueExcelFile.add_worksheet('sheet1')
    sheet.set_column('B:Z',35)
    q = stateActionValue.calStateActionValue(sheet, q, policy, theta, gamma, initPolicyPi, printFlag, outputMidFlag)
    actionValueExcelFile.close()
    print('\n\n确定环境机器人寻宝任务的状态动作值：')
    print("不可达状态：", end = '  ')
    for unreachstate in env.unreachState:
        print(unreachstate, end = '  ')
    print("\n终止状态：", end = '  ')
    for endstate in env.endState:
        print(endstate, end = '  ')
    print("\n特殊状态：", end = '  ')
    for typestate in env.typeState:
        print(typestate, end = '  ')
    print("\n是否允许冲出边界的动作", end = '  ')
    if edgeFlag:
        print("允许\n")
    else:
        print("不允许\n")
    stateActionValue.printActionValue(q)

迭代过程：
第1轮迭代状态动作值
状态       上       下       左       右
( 0, 0)    *.***    0.000    *.***    0.000
( 0, 1)    *.***    0.000    0.000    0.000
( 0, 2)    *.***    0.000    0.000    0.000
( 0, 3)    *.***    0.000    0.000    0.000
( 0, 4)    *.***    3.000    0.000    *.***
( 1, 0)    0.000    0.000    *.***    0.000
( 1, 1)    0.000    0.000    0.000    0.000
( 1, 2)    0.000  -10.000    0.000    0.000
( 1, 3)    0.000    0.000   -2.000    3.000
( 1, 4)    *.***    *.***    *.***    *.***
( 2, 0)    0.000    0.000    *.***    0.000
( 2, 1)    0.000    0.000    0.000  -10.000
( 2, 2)    *.***    *.***    *.***    *.***
( 2, 3)    0.200    0.000   -9.960    0.000
( 2, 4)    3.000    0.000   -1.952    *.***
( 3, 0)    0.000    1.000    *.***    0.000
( 3, 1)   -2.000    0.000    0.267    0.000
( 3, 2)  -10.000    0.000   -0.347    0.000
( 3, 3)   -1.952    0.000   -2.069    0.000
( 3, 4)    0.279    0.000   -0.804    *.***
( 4, 0)    *.***    *.***    *.***    *.***
( 4, 1)   -0.347    *.**

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;代码2:基于随机环境动作值函数的策略评估代码实现<br>

In [ ]:
# 过程----随机环境状态动作值函数评估
from mdpModel import MdpModelRandom, Tools
import numpy as np
class StateActionValueRandomEnv(MdpModelRandom, Tools):
    def calStateActionValueRandomEnv(self, sheet, q, policy, theta, gamma, printFlag, outputMidFlag):               # 阈值，设置为一个较小的实值
        self.q = q
        self.printFlag = printFlag
        self.outputMidFlag = outputMidFlag
        tao = 0                                          # 迭代轮次
        while True:
            delta = 0                                    # 相对应状态动作值函数差值的最小值的绝对值，初始化为0
            qBackup = self.q.copy()
            for s in self.stateSpace():
                row, col = s
                actionNumber = 0
                for actionPi in policy[row * env.width + col]:
                    if actionPi > 0:
                        act = self.actionSpace()
                        a = act[actionNumber]                
                        stateActionValue = 0                   
                        for s1 in self.stateSpace():
                            row1, col1 = s1
                            actionNumber1 = 0
                            tempValue = 0.0
                            
                            for actionPi1 in policy[row1 * env.width + col1]:
                                pi = actionPi1
                                tempValue = tempValue + pi * self.q[row1 * env.width + col1][actionNumber1]
                                actionNumber1 += 1
                            
                            if not(isinstance(self.transferFunction(s, a, s1), tuple)):
                                stateActionValue = stateActionValue + self.transferFunction(s, a, s1) * ((self.rewardFunction(s, a, s1) + gamma * tempValue))
                            else:
                                noReach = self.transferFunction(s, a, s1)
                                stateActionValue = stateActionValue + noReach[0] * ((self.rewardFunction(s, a, s1) + gamma * tempValue))
                                stateActionValue = stateActionValue + noReach[1] * (self.env.defaultReward + gamma * tempValue)
                                stateActionValue = stateActionValue + noReach[2] * ((self.rewardFunction(s, a, s1) + gamma * tempValue))
                        self.q[row * env.width + col][actionNumber] = stateActionValue
                        delta = max(delta, abs(self.q[row * env.width + col][actionNumber] - qBackup[row * env.width + col][actionNumber]))
                    else:
                        self.q[row * env.width + col][actionNumber] = -99999
                    actionNumber += 1
            tao += 1        
            if self.printFlag:
                print('第%d轮迭代状态动作值'%tao)
                self.printActionValue(self.q)
            if self.outputMidFlag:
                self.actionValueOutputExcel(sheet, self.q, tao)
            if delta < theta:
                break
        return self.q   

In [ ]:
# 结果----随机环境状态动作值函数评估结果
if __name__ == '__main__':
    stateActionValue = StateActionValueRandomEnv(env)
    initPolicyPi  = stateActionValue.stateActionSpacePi()
    q = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
    policy = initPolicyPi.copy()
    printFlag = True
    theta = 0.0001
    gamma = 0.8
    printFlag = True                                                  #是否打印评估过程
    outputMidFlag = True                                              #是否输出中间过程到Excel表
    if printFlag:
        print("迭代过程：")
    actionValueExcelFile = xlsxwriter.Workbook(r'.\resultTable\表4.2 初始策略下随机环境机器人寻宝任务的动作值函数评估过程表.xlsx')
    sheet = actionValueExcelFile.add_worksheet('sheet1')
    sheet.set_column('B:Z',35)
    stateActionValue.actionValueOutputExcel(sheet, q, 0)
    q = stateActionValue.calStateActionValueRandomEnv(sheet, q, policy, theta, gamma, printFlag, outputMidFlag)
    actionValueExcelFile.close()
    print('\n\n确定环境机器人寻宝任务的状态动作值：')
    print("不可达状态：", end = '  ')
    for unreachstate in env.unreachState:
        print(unreachstate, end = '  ')
    print("\n终止状态：", end = '  ')
    for endstate in env.endState:
        print(endstate, end = '  ')
    print("\n特殊状态：", end = '  ')
    for typestate in env.typeState:
        print(typestate, end = '  ')
    print("\n是否允许冲出边界的动作", end = '  ')
    if edgeFlag:
        print("允许\n")
    else:
        print("不允许\n")
    stateActionValue.printActionValue(q)

迭代过程：
第1轮迭代状态动作值
状态       上       下       左       右
( 0, 0)    *.***    0.000    *.***    0.000
( 0, 1)    *.***    0.000    0.000    0.000
( 0, 2)    *.***    0.000    0.000    0.000
( 0, 3)    *.***    0.000    0.000    0.000
( 0, 4)    *.***    2.400    0.192    *.***
( 1, 0)    0.000    0.000    *.***    0.000
( 1, 1)    0.000    0.000    0.000    0.000
( 1, 2)   -0.500   -8.095   -0.258   -0.266
( 1, 3)    0.000    0.000   -1.309    2.270
( 1, 4)    *.***    *.***    *.***    *.***
( 2, 0)    0.000    0.000    *.***    0.000
( 2, 1)    0.000    0.000   -0.500   -8.095
( 2, 2)    *.***    *.***    *.***    *.***
( 2, 3)    0.154    0.014   -7.968   -0.812
( 2, 4)    2.400    0.246   -1.237    *.***
( 3, 0)    0.050    0.802    *.***    0.045
( 3, 1)   -1.375   -0.127    0.146   -0.029
( 3, 2)   -8.000   -0.820   -0.486   -0.293
( 3, 3)   -1.378   -0.127   -1.581   -0.189
( 3, 4)    0.301    0.031   -0.506    *.***
( 4, 0)    *.***    *.***    *.***    *.***
( 4, 1)   -0.222    *.**

第9轮迭代状态动作值
状态       上       下       左       右
( 0, 0)    *.***   -0.748    *.***   -0.724
( 0, 1)    *.***   -1.387   -0.636   -0.935
( 0, 2)    *.***   -2.704   -0.786   -0.278
( 0, 3)    *.***   -0.503   -0.766    0.813
( 0, 4)    *.***    2.620    0.123    *.***
( 1, 0)   -0.646   -1.047    *.***   -1.398
( 1, 1)   -1.031   -2.964   -1.049   -2.782
( 1, 2)   -1.931  -11.031   -1.754   -1.031
( 1, 3)   -0.331   -2.405   -2.460    2.151
( 1, 4)    *.***    *.***    *.***    *.***
( 2, 0)   -0.846   -0.512    *.***   -2.919
( 2, 1)   -1.829   -1.729   -2.087  -11.267
( 2, 2)    *.***    *.***    *.***    *.***
( 2, 3)   -0.992   -1.614  -10.748   -1.118
( 2, 4)    2.368   -0.247   -2.326    *.***
( 3, 0)   -0.919    0.686    *.***   -1.222
( 3, 1)   -2.939   -0.686   -0.693   -2.902
( 3, 2)  -11.223   -2.059   -1.728   -1.724
( 3, 3)   -2.569   -0.973   -2.918   -0.774
( 3, 4)   -0.141   -0.457   -1.256    *.***
( 4, 0)    *.***    *.***    *.***    *.***
( 4, 1)   -1.231    *.***    0

第17轮迭代状态动作值
状态       上       下       左       右
( 0, 0)    *.***   -0.825    *.***   -0.797
( 0, 1)    *.***   -1.447   -0.696   -0.986
( 0, 2)    *.***   -2.746   -0.830   -0.308
( 0, 3)    *.***   -0.525   -0.795    0.799
( 0, 4)    *.***    2.618    0.107    *.***
( 1, 0)   -0.708   -1.100    *.***   -1.454
( 1, 1)   -1.078   -3.008   -1.095   -2.823
( 1, 2)   -1.963  -11.067   -1.788   -1.054
( 1, 3)   -0.350   -2.427   -2.483    2.148
( 1, 4)    *.***    *.***    *.***    *.***
( 2, 0)   -0.892   -0.543    *.***   -2.961
( 2, 1)   -1.865   -1.762   -2.120  -11.297
( 2, 2)    *.***    *.***    *.***    *.***
( 2, 3)   -1.007   -1.636  -10.766   -1.132
( 2, 4)    2.365   -0.263   -2.339    *.***
( 3, 0)   -0.949    0.681    *.***   -1.252
( 3, 1)   -2.966   -0.707   -0.712   -2.933
( 3, 2)  -11.255   -2.084   -1.749   -1.747
( 3, 3)   -2.585   -0.993   -2.938   -0.791
( 3, 4)   -0.151   -0.472   -1.270    *.***
( 4, 0)    *.***    *.***    *.***    *.***
( 4, 1)   -1.250    *.***    

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;代码3:基于确定环境动作值函数的策略改进代码实现<br>

In [ ]:
import numpy as np
from mdpModel import Tools
class StateActPolicyImprove(StateActionValue, Tools):
    def stateActPolicyEval(self, sheet, q, policy, theta, gamma, initPolicyPi, printFlag, outputMidFlag):
        self.q = self.calStateActionValue(sheet, q, policy, theta, gamma, initPolicyPi, printFlag, outputMidFlag)
        return self.q

    def stateActPolicyImprove(self, q):
        newPolicy = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
        for s in env.stateSpace:
            row, col = s
            qMax = -9999
            actNum = 0
            stateActionSpace = self.stateActionSpacePi()
            for action in stateActionSpace[row * env.width + col]:
                qMax = max(qMax, q[row * env.width + col][actNum])
                actNum += 1
            actNum = 0
            actCount = list(stateActionSpace[row * env.width + col]).count(1)
            for action in stateActionSpace[row * env.width + col]:
                if action > 0:
                    if abs(q[row * env.width + col][actNum] - qMax) < 0.0001:
                        newPolicy[row * env.width + col][actNum] = 1
                actNum += 1
            actCount = list(newPolicy[row * env.width + col]).count(1)
            actNum = 0
            for act in list(newPolicy[row * env.width + col]):
                if act == 1:
                    newPolicy[row * env.width + col][actNum] = 1. /actCount
                actNum += 1
        return newPolicy

In [ ]:
#面向确定环境机器人寻宝任务的动作值函数的迭代更新过程
if __name__ == '__main__':
    actionPolicyValueIterExcelFile = xlsxwriter.Workbook(r'.\resultTable\表4.4 面向确定环境机器人寻宝任务的动作值函数的迭代更新过程表.xlsx')
    sheet = actionPolicyValueIterExcelFile.add_worksheet('sheet1')
    sheet.set_column('B:Z',35)
    print("策略迭代过程：")
    stateActPolicyImprove = StateActPolicyImprove(env)
    q = np.array(([[0.0 for row in range(env.actionCount)] for col in range(env.height * env.width)]))
    theta = 0.0001
    gamma = 0.8
    initPolicyPi = stateActPolicyImprove.stateActionSpacePi()
    newPolicy = stateActPolicyImprove.stateActionSpacePi()
    printFlag = False
    outputMidFlag = False
    policyStable = True
    l = 0
    print('初始动作值：')
    stateActPolicyImprove.printActionValue(q)
    print('初始策略：')
    stateActPolicyImprove.printActionValue(newPolicy)
    stateActPolicyImprove.actionPolicyValueOutputExcel(sheet, q, newPolicy, l)
    while policyStable:
        oldPolicy = newPolicy.copy()
        q = stateActPolicyImprove.stateActPolicyEval(sheet, q, newPolicy, theta, gamma, initPolicyPi, printFlag, outputMidFlag)
        l += 1
        print('第%2d轮策略迭代：'%l)
        print('动作值(%2d)：'%l)
        stateActPolicyImprove.printActionValue(q)
        print('策略(%2d)：'%l)
        newPolicy = stateActPolicyImprove.stateActPolicyImprove(q)
        stateActPolicyImprove.actionPolicyValueOutputExcel(sheet, q, newPolicy, l)
        stateActPolicyImprove.printActionValue(newPolicy)
        if (newPolicy == oldPolicy).all():
            policyStable = False
    actionPolicyValueIterExcelFile.close()

策略迭代过程：
初始动作值：
状态       上       下       左       右
( 0, 0)    0.000    0.000    0.000    0.000
( 0, 1)    0.000    0.000    0.000    0.000
( 0, 2)    0.000    0.000    0.000    0.000
( 0, 3)    0.000    0.000    0.000    0.000
( 0, 4)    0.000    0.000    0.000    0.000
( 1, 0)    0.000    0.000    0.000    0.000
( 1, 1)    0.000    0.000    0.000    0.000
( 1, 2)    0.000    0.000    0.000    0.000
( 1, 3)    0.000    0.000    0.000    0.000
( 1, 4)    0.000    0.000    0.000    0.000
( 2, 0)    0.000    0.000    0.000    0.000
( 2, 1)    0.000    0.000    0.000    0.000
( 2, 2)    0.000    0.000    0.000    0.000
( 2, 3)    0.000    0.000    0.000    0.000
( 2, 4)    0.000    0.000    0.000    0.000
( 3, 0)    0.000    0.000    0.000    0.000
( 3, 1)    0.000    0.000    0.000    0.000
( 3, 2)    0.000    0.000    0.000    0.000
( 3, 3)    0.000    0.000    0.000    0.000
( 3, 4)    0.000    0.000    0.000    0.000
( 4, 0)    0.000    0.000    0.000    0.000
( 4, 1)    0.000    0.000 

状态       上       下       左       右
( 0, 0)    0.000    0.500    0.000    0.500
( 0, 1)    0.000    0.500    0.000    0.500
( 0, 2)    0.000    0.500    0.000    0.500
( 0, 3)    0.000    0.500    0.000    0.500
( 0, 4)    0.000    1.000    0.000    0.000
( 1, 0)    0.000    0.000    0.000    1.000
( 1, 1)    0.000    0.000    0.000    1.000
( 1, 2)    0.000    0.000    0.000    1.000
( 1, 3)    0.000    0.000    0.000    1.000
( 1, 4)    0.000    0.000    0.000    0.000
( 2, 0)    0.500    0.000    0.000    0.500
( 2, 1)    1.000    0.000    0.000    0.000
( 2, 2)    0.000    0.000    0.000    0.000
( 2, 3)    0.500    0.000    0.000    0.500
( 2, 4)    1.000    0.000    0.000    0.000
( 3, 0)    0.000    1.000    0.000    0.000
( 3, 1)    0.500    0.000    0.000    0.500
( 3, 2)    0.000    0.000    0.000    1.000
( 3, 3)    0.500    0.000    0.000    0.500
( 3, 4)    1.000    0.000    0.000    0.000
( 4, 0)    0.000    0.000    0.000    0.000
( 4, 1)    0.000    0.000    1.000    0.0

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;动画效果

In [ ]:
import sys
import os
from time import sleep
import signal
import numpy as np
import random

def CtrlCHandler(signum, frame):
    env.render(close=True)    

def rendering(renderFlag, optimPolicy):
    if renderFlag:
        episodeNum = 10
        for i_episode in range(episodeNum):
            observation = env.reset()
            for t in range(100):
                signal.signal(signal.SIGINT, CtrlCHandler)
                env.render()
                sleep(0.3)
                print('state',observation)
                row, col = observation
                actionlist = optimPolicy[row * env.width + col]
                a = -1
                while a <= 0:
                    aIndex = random.randrange(len(actionlist))
                    a = actionlist[aIndex]
                observation, reward, done, info = env.step(aIndex, actionAllList[:actionCount])
                if done:
                    env.render()
                    print("Episode finished after {} timesteps".format(t+1))
                    break
            sleep(2)
        env.render(close = True)
if __name__ =='__main__':
    rendering(True, newPolicy)          

state (3, 3)
state (3, 4)
state (2, 4)
Episode finished after 3 timesteps
state (0, 4)
Episode finished after 1 timesteps
state (0, 4)
Episode finished after 1 timesteps
state (0, 1)
state (0, 2)
state (0, 3)
state (1, 3)
Episode finished after 4 timesteps
state (0, 1)
state (1, 1)
state (1, 2)
state (1, 3)
Episode finished after 4 timesteps
state (3, 0)
Episode finished after 1 timesteps
state (4, 3)
state (3, 3)
state (2, 3)
state (2, 4)
Episode finished after 4 timesteps
state (2, 0)
state (2, 1)
state (1, 1)
state (1, 2)
state (1, 3)
Episode finished after 5 timesteps
state (3, 4)
state (2, 4)
Episode finished after 2 timesteps
state (0, 3)
state (0, 4)
Episode finished after 2 timesteps


&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;代码4:基于确定环境状态值函数的策略迭代代码实现<br>

In [ ]:
from mdpModel import MdpModel, Tools
import numpy as np
class StateValue(MdpModel, Tools):
    def calStateValue(self, sheet, v, policy, theta, gamma, printFlag, outputMidFlag):               # 阈值，设置为一个较小的实值
        self.outputMidFlag = outputMidFlag   
        self.v = v      # 策略评估的状态值函数
        tao = 0                                                                # 迭代轮次
        while True:
            delta = 0                                                          # 相对应状态值函数差值的最小值的绝对值，初始化为0
            vBackup = self.v.copy()               # 暂存未更新状态值函数的数据，以便与更新后的状态值函数比较，确定退出的条件
            for s in self.stateSpace():
                row, col = s
                stateValue = 0    # 当前位置的状态值
                actionNum = 0     # 进行到了哪步动作
                for actionPi in policy[row * env.width + col]:
                    if actionPi > 0:                
                        act = self.actionSpace()
                        a = act[actionNum]
                        env.state = s
                        s1, _, _, _ = env.step(actionNum, self.actionSpace())
                        pi = actionPi
                        p = self.transferFunction(s, a, s1)
                        r = self.rewardFunction(s, a)
                        row1, col1 = s1
                        stateValue = stateValue + pi * p * (r + gamma * self.v[row1][col1])
                    actionNum += 1
                self.v[row][col] = stateValue
                delta = max(delta, abs(self.v[row][col] - vBackup[row][col]))
            tao += 1
            if printFlag:
                print('第%d轮迭代状态值'%tao)
                self.printStateValue(self.v)
                print('——————————————————————————————————————')
            if self.outputMidFlag:
                self.stateValueOutputExcel(sheet, self.v, tao)
            if delta < theta:
                break
        return self.v   

In [ ]:
# 初始策略下确定环境机器人寻宝任务的状态值函数评估过程主程序
if __name__ == '__main__':
    stateValue = StateValue(env)
    initPolicy = stateValue.stateActionSpacePi()   # 初始策略分布
    v = np.array(([[0.0 for col in range(env.width)] for row in range(env.height)]))
    theta = 0.0001
    gamma = 0.8
    printFlag = True                                                  #是否打印评估过程
    outputMidFlag = True                                              #是否输出中间过程到Excel表
    if printFlag:
        print("迭代过程：")
    if outputMidFlag:
        stateValueExcelFile = xlsxwriter.Workbook(r'.\resultTable\图4.2 初始策略下确定环境机器人寻宝任务的状态值函数评估过程表.xlsx')
        sheet = stateValueExcelFile.add_worksheet('sheet1')
        sheet.set_column('B:Z',12)
        stateValue.stateValueOutputExcel(sheet, v, 0)
    v = stateValue.calStateValue(sheet, v, initPolicy, theta, gamma, printFlag, outputMidFlag)
    print("迭代结果：")
    print('确定环境机器人寻宝任务的状态值：')
    print("不可达状态：", end = '  ')
    for unreachstate in env.unreachState:
        print(unreachstate, end = '  ')
    print("\n终止状态：", end = '  ')
    for endstate in env.endState:
        print(endstate, end = '  ')
    print("\n特殊状态：", end = '  ')
    for typestate in env.typeState:
        print(typestate, end = '  ')
    print("\n是否允许冲出边界的动作", end = '  ')
    if edgeFlag:
        print("允许\n")
    else:
        print("不允许\n")
    stateValue.printStateValue(v)
    stateValueExcelFile.close()

迭代过程：
第1轮迭代状态值
    0.000     0.000     0.000     0.000     1.500 
    0.000     0.000    -2.500     0.250     0.000 
    0.000    -2.500     0.000    -2.450     0.347 
    0.333    -0.433    -2.587    -1.007    -0.176 
    0.000     0.218    -0.632    -0.437    -0.245 
——————————————————————————————————————
第2轮迭代状态值
    0.000     0.000    -0.667     0.289     1.616 
    0.000    -1.000    -3.283    -0.339     0.000 
   -0.578    -3.402     0.000    -3.190     0.102 
    0.064    -1.141    -3.573    -1.475    -0.432 
    0.000    -0.140    -1.107    -0.754    -0.474 
——————————————————————————————————————
第3轮迭代状态值
    0.000    -0.444    -0.917     0.096     1.538 
   -0.421    -1.510    -3.710    -0.611     0.000 
   -1.002    -3.911     0.000    -3.535    -0.058 
   -0.238    -1.573    -4.046    -1.753    -0.609 
    0.000    -0.381    -1.382    -0.962    -0.629 
——————————————————————————————————————
第4轮迭代状态值
   -0.346    -0.740    -1.161    -0.062     1.475 
   -0.762    -1.825    -3

In [ ]:
import numpy as np
class StatePolicyImprove(StateValue, Tools):
    def statePolicyEval(self, sheet, v, policy, theta, gamma, printFlag, outputMidFlag):
        self.v = self.calStateValue(sheet, v, policy, theta, gamma, printFlag, outputMidFlag)  # 策略评估的状态值函数
        return self.v
    
    def statePolicyImprove(self, v):  # 策略改进
        newPolicy = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
        qValue = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
        for s in self.stateSpace():
            row, col = s
            qMax = -9999
            actionNum = 0
            stateActionSpace = self.stateActionSpacePi()
            # 根据最大动作值，进行策略改进
            # 计算每个动作下的q值，同时记录最大动作值
            for action in stateActionSpace[row * env.width + col]:
                stateValue = 0
                if action > 0:
                    act = self.actionSpace()
                    a = act[actionNum]
                    env.state = s
                    s1, _, _, _ = env.step(actionNum, act)
                    p = self.transferFunction(s, a, s1)
                    r = self.rewardFunction(s, a)
                    row1, col1 = s1
                    qValue[row * env.width + col][actionNum] = qValue[row * env.width + col][actionNum] + p * (r + gamma * self.v[row1][col1])
                else:
                    qValue[row * env.width + col][actionNum] = -99999
                qMax = max(qMax, qValue[row * env.width + col][actionNum])
                actionNum += 1
            # 寻找同一状态下与最大动作值，值一样的动作，使其选择的概率为1
            actNum = 0
            actCount = list(stateActionSpace[row * env.width + col]).count(1)
            for action in stateActionSpace[row * env.width + col]:
                if action > 0:
                    if round(qValue[row * env.width + col][actNum], 5) == round(qMax, 5):
                        newPolicy[row * env.width + col][actNum] = 1
                actNum += 1
            # 根据最大动作值数，更新策略    
            actCount = list(newPolicy[row * env.width + col]).count(1)  # 可采取动作的个数
            actNum = 0
            for act in list(newPolicy[row * env.width + col]):
                if act == 1:
                    newPolicy[row * env.width + col][actNum] = 1. /actCount
                actNum += 1
        return newPolicy, qValue

In [ ]:
if __name__ == '__main__':
    statePolicyImprove = StatePolicyImprove(env)
    initPolicyPi = statePolicyImprove.stateActionSpacePi()
    stateActionPolicyValueIterExcelFile = xlsxwriter.Workbook(r'.\resultTable\表4.3 面向确定环境机器人寻宝任务的状态值迭代更新过程表.xlsx')
    sheet = stateActionPolicyValueIterExcelFile.add_worksheet('sheet1')
    sheet.set_column('B:Z',35)
    print("策略迭代过程：")
    theta = 0.0001
    gamma = 0.8
    newPolicy = initPolicyPi.copy()
    printFlag = False
    outputMidFlag = False
    policyStable = True
    l = 0
    v = np.array(([[0.0 for col in range(env.width)] for row in range(env.height)]))
    q = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
    statePolicyImprove.stateActionPolicyValueOutputExcel(sheet, v, q, initPolicyPi, l)
    while policyStable:
        oldPolicy = newPolicy.copy()
        v = statePolicyImprove.statePolicyEval(sheet, v, newPolicy, theta, gamma, printFlag, outputMidFlag)
        statePolicyImprove.printStateValue(v)
        newPolicy, qValue = statePolicyImprove.statePolicyImprove(v)
        statePolicyImprove.printActionValue(newPolicy)
        l += 1
        statePolicyImprove.stateActionPolicyValueOutputExcel(sheet, v, qValue, newPolicy, l)
        if (newPolicy == oldPolicy).all():  # 跳出条件是新旧策略完全相同
            policyStable = False
    stateActionPolicyValueIterExcelFile.close()

策略迭代过程：
   -1.110    -1.359    -1.615    -0.329     1.368 
   -1.417    -2.372    -4.369    -0.987     0.000 
   -1.831    -4.716     0.000    -3.987    -0.300 
   -0.731    -2.162    -4.649    -2.160    -0.887 
    0.000    -0.716    -1.772    -1.280    -0.867 
状态       上       下       左       右
( 0, 0)    0.000    0.000    0.000    1.000
( 0, 1)    0.000    0.000    1.000    0.000
( 0, 2)    0.000    0.000    0.000    1.000
( 0, 3)    0.000    0.000    0.000    1.000
( 0, 4)    0.000    1.000    0.000    0.000
( 1, 0)    1.000    0.000    0.000    0.000
( 1, 1)    1.000    0.000    0.000    0.000
( 1, 2)    0.000    0.000    0.000    1.000
( 1, 3)    0.000    0.000    0.000    1.000
( 1, 4)    0.000    0.000    0.000    0.000
( 2, 0)    0.000    1.000    0.000    0.000
( 2, 1)    0.000    0.000    1.000    0.000
( 2, 2)    0.000    0.000    0.000    0.000
( 2, 3)    0.000    0.000    0.000    1.000
( 2, 4)    1.000    0.000    0.000    0.000
( 3, 0)    0.000    1.000    0.000    0.00

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;代码5:汽车租赁问题。
    
&emsp;&emsp;杰克经营两家异地的汽车租赁场（A，B租赁场），每天都会有客户来租车。如果每个租赁场有可供外租的汽车，每租出一辆汽车，杰克都会获得10美元的租金。为了保证每个租赁场有足够的车辆可租，每天晚上杰克会在两个租赁场之间移动车辆，每辆车的移车费用为2美元。假设每个租赁场的租车和还车的数量是一个泊松随机量，即期望数量$n$的概率为$\frac{\lambda^n}{n!}e^{-\lambda}$ ，其中$\lambda$为期望值。假设在两个租赁场租车的$\lambda$分别是$\lambda_{A1}$=4,$\lambda_{B1}$=3， ；还车的$\lambda$分别为$\lambda_{A2}$=2,$\lambda_{B2}$=3。<br>
&emsp;&emsp;为了简化问题，给定3个假设：（1）任何一个租赁场车辆总数不超过20辆车；（2）当天还回的车辆第2天才能出租。（3）两个租车场之间每天最多可移车数量为5辆。<br>
&emsp;&emsp;利用策略迭代方法，在折扣率$\gamma$=0.9时，计算两个租赁场点之间的最优移车策略。<br>
&emsp;&emsp;将汽车租赁问题描述为一个持续的有限MDP模型。其中时刻$t$按天计算；状态为每天租还车结束时，两个租赁场的车辆数；动作为每天晚上在两个租赁场之间移动的车辆数目。<br>
&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;结果：
<center><img src="./image/绘图4.6汽车租赁.jpg" ></center><br>

 &emsp;&emsp;该问题需要从一个确定策略出发进行策略迭代。本实验的初始策略为：在任何状态下，不考虑两个租赁场的需求，选择不移动车辆（动作为0）。然后进行策略评估，并根据值函数进行策略改进。如此反复，直到策略收敛，找到最优策略。图中（5）即为最优策略图。<br>

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;Jack的汽车租赁程序代码实现：

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import poisson
%matplotlib inline
plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
plt.rcParams['axes.unicode_minus']=False

matplotlib.use('Agg')

# 每个租赁场最多车辆数
MAX_CARS = 20

# 每天晚上最多移动的车辆数
MAX_MOVE_OF_CARS = 5

# A租赁场租出的期望值
RENTAL_REQUEST_A_LOC = 4

# B租赁场租出的期望值
RENTAL_REQUEST_B_LOC = 3

# A租赁场还回的期望值
RETURNS_A_LOC = 2

# B租赁场还回的期望值
RETURNS_B_LOC = 3

DISCOUNT = 0.9

# 租出一辆车的收益
RENTAL_CREDIT = 10

# 挪车的费用
MOVE_CAR_COST = 2

# 所有可能的动作
actions = np.arange(-MAX_MOVE_OF_CARS, MAX_MOVE_OF_CARS + 1)

# 泊松分布上界
# If n is greater than this value, then the probability of getting n is truncated to 0
POISSON_UPPER_BOUND = 11

# 泊松分布的概率
# @lam: lambda <= 10
poisson_cache = dict()

def poisson_probability(n, lam):
    global poisson_cache
    key = n * 10 + lam
    if key not in poisson_cache:
        poisson_cache[key] = poisson.pmf(n, lam)
    return poisson_cache[key]

def expected_return(state, action, state_value, constant_returned_cars):
    """
    @state: [A租赁场当前车辆数, B租赁场当前车辆数]
    @action: 从A移动到B的车辆数为正,从B移动到A的车辆数为负
    @stateValue: state value matrix
    @constant_returned_cars:  if set True, model is simplified such that
    the # of cars returned in daytime becomes constant
    rather than a random value from poisson distribution, which will reduce calculation time
    and leave the optimal policy/value state matrix almost the same
    """
    # 初始化总的回报
    returns = 0.0

    # 移动车辆的费用
    returns -= MOVE_CAR_COST * abs(action)

    # 车辆移完后两个租赁场的车辆情况
    NUM_OF_CARS_A_LOC = min(state[0] - action, MAX_CARS)
    NUM_OF_CARS_B_LOC = min(state[1] + action, MAX_CARS)

    # 所有可能的租车情况
    for rental_request_a_loc in range(POISSON_UPPER_BOUND):
        for rental_request_b_loc in range(POISSON_UPPER_BOUND):
            # 所有可能的租车概率
            prob = poisson_probability(rental_request_a_loc, RENTAL_REQUEST_A_LOC) * \
                poisson_probability(rental_request_b_loc, RENTAL_REQUEST_B_LOC)

            num_of_cars_a_loc = NUM_OF_CARS_A_LOC
            num_of_cars_b_loc = NUM_OF_CARS_B_LOC

            # 有效的租车数，可以租出的车辆数不大于实际车辆数
            valid_rental_a_loc = min(num_of_cars_a_loc, rental_request_a_loc)
            valid_rental_b_loc = min(num_of_cars_b_loc, rental_request_b_loc)

            # 租车获得的收益
            reward = (valid_rental_a_loc + valid_rental_b_loc) * RENTAL_CREDIT
            num_of_cars_a_loc -= valid_rental_a_loc
            num_of_cars_b_loc -= valid_rental_b_loc

            if constant_returned_cars:
                # get returned cars, those cars can be used for renting tomorrow
                returned_cars_a_loc = RETURNS_A_LOC
                returned_cars_b_loc = RETURNS_B_LOC
                num_of_cars_a_loc = min(num_of_cars_a_loc + returned_cars_a_loc, MAX_CARS)
                num_of_cars_b_loc = min(num_of_cars_b_loc + returned_cars_b_loc, MAX_CARS)
                returns += prob * (reward + DISCOUNT * state_value[num_of_cars_a_loc, num_of_cars_b_loc])
            else:
                for returned_cars_a_loc in range(POISSON_UPPER_BOUND):
                    for returned_cars_b_loc in range(POISSON_UPPER_BOUND):
                        prob_return = poisson_probability(
                            returned_cars_a_loc, RETURNS_A_LOC) * poisson_probability(returned_cars_b_loc, RETURNS_B_LOC)
                        num_of_cars_a_loc_ = min(num_of_cars_a_loc + returned_cars_a_loc, MAX_CARS)
                        num_of_cars_b_loc_ = min(num_of_cars_b_loc + returned_cars_b_loc, MAX_CARS)
                        prob_ = prob_return * prob
                        returns += prob_ * (reward + DISCOUNT *
                                            state_value[num_of_cars_a_loc_, num_of_cars_b_loc_])
    return returns


def figure_4_2(constant_returned_cars = True):
    value = np.zeros((MAX_CARS + 1, MAX_CARS + 1))
    policy = np.zeros(value.shape, dtype=np.int)

    iterations = 0
    _, axes = plt.subplots(2, 3, figsize=(40, 20))
    plt.subplots_adjust(wspace=0.1, hspace=0.2)
    axes = axes.flatten()
    number = 0
    print('Round %d iteration:\n '%number)
    print('Value function\n')
    for i in range(MAX_CARS + 1):
        print('%4d'%(20 - i), end = ' ')
        for j in range(MAX_CARS + 1):
            print('%4d'%value[20 - i, j], end = ' ')
        print('\n')
    print('    ', end =' ')
    for k in range(21):
        print('%4d'%k, end = ' ')
    print('\n\n')
    print('Policy\n')
    for i in range(MAX_CARS + 1):
        print('%4d'%(20 - i), end = ' ')
        for j in range(MAX_CARS + 1):
            print('%4d'%policy[20 - i, j], end = ' ')
        print('\n')
    print('    ', end =' ')
    for k in range(21):
        print('%4d'%k, end = ' ')
    print('\n')
    while True:
        fig = sns.heatmap(np.flipud(policy), cmap="YlGnBu", vmax=5, vmin=-5,ax=axes[iterations],center=0)
        fig.set_ylabel('A rental site ',fontsize=30)
        fig.set_yticks(list(reversed(range(MAX_CARS + 1))))
        fig.set_xlabel('B rental site ', fontsize=30)
        fig.set_title('policy {}'.format(iterations), fontsize=30)

        # 策略评估
        print('Round %d iteration:\n '%number)
        while True:
            old_value = value.copy()
            for i in range(MAX_CARS + 1):
                for j in range(MAX_CARS + 1):
                    new_state_value = expected_return([i, j], policy[i, j], value, constant_returned_cars)
                    value[i, j] = new_state_value
            max_value_change = abs(old_value - value).max()
            print('max value change {}'.format(max_value_change))
            if max_value_change < 1e-4:
                print('Value function:')
                for i in range(MAX_CARS + 1):
                    print('%4d'%(20 - i), end = ' ')
                    for j in range(MAX_CARS + 1):
                        print('%4d'%value[20 - i, j], end = ' ')
                    print('\n')
                print('    ', end =' ')
                for k in range(21):
                    print('%4d'%k, end = ' ')
                print('\n\n')
                break
        number += 1 
        # 策略改进
        policy_stable = True
        for i in range(MAX_CARS + 1):
            for j in range(MAX_CARS + 1):
                old_action = policy[i, j]
                action_returns = []
                for action in actions:
                    if (0 <= action <= i) or (-j <= action <= 0):
                        action_returns.append(expected_return([i, j], action, value, constant_returned_cars))
                    else:
                        action_returns.append(-np.inf)
                new_action = actions[np.argmax(action_returns)]
                policy[i, j] = new_action
                if policy_stable and old_action != new_action:
                    policy_stable = False
        print('policy stable {}'.format(policy_stable))
        print('Policy')
        for i in range(MAX_CARS + 1):
            print('%4d'%(20 - i), end = ' ')
            for j in range(MAX_CARS + 1):
                print('%4d'%policy[20 - i, j], end = ' ')
            print('\n')
        print('    ', end =' ')
        for k in range(21):
            print('%4d'%k, end = ' ')
        print('\n')

        if policy_stable:
            fig = sns.heatmap(np.flipud(value), cmap="YlGnBu", ax=axes[-1])
            fig.set_ylabel('Number of cars in the A rental site', fontsize=30)
            fig.set_yticks(list(reversed(range(MAX_CARS + 1))))
            fig.set_xlabel('Number of cars in the B rental site', fontsize=30)
            fig.set_title('optimal value', fontsize=30)
            break

        iterations += 1

    plt.savefig('images/figure_4_2.png')
    plt.close()


if __name__ == '__main__':
    figure_4_2()

Round 0 iteration:
 
Value function

  20    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  19    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  18    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  17    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  16    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  15    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  14    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  13    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  12    0    0    0    0    0    0    0    0    0    0    0    0   

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei


Round 0 iteration:
 
max value change 242.69734618420745
max value change 124.0988199892968
max value change 70.32718327075344
max value change 48.94488247898738
max value change 41.488250014681086
max value change 35.03521693218764
max value change 29.64178677090021
max value change 25.8308384863233
max value change 22.396739990982212
max value change 19.350456685914367
max value change 16.674507474144264
max value change 14.339324865231788
max value change 12.311095840284509
max value change 10.555662149529894
max value change 9.040461785396815
max value change 7.73544365819572
max value change 6.613426120390784
max value change 5.650150793260025
max value change 4.824171763458878
max value change 4.116660390788297
max value change 3.5111723929849177
max value change 2.9934043848315355
max value change 2.5509554550628764
max value change 2.173102354529874
max value change 1.850592597168145
max value change 1.5754571756690439
max value change 1.340843032431053
max value change 1.14086

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Generic family 'sans-serif' not found because none of the following families were found: SimHei


Round 1 iteration:
 
max value change 36.70275351094716
max value change 26.16533214415773
max value change 9.231581077044211
max value change 1.7893569246647303
max value change 0.8741252794181946
max value change 0.6880541444028836
max value change 0.5886080231504138
max value change 0.5015654929139828
max value change 0.4266769921516129
max value change 0.36269049975646794
max value change 0.3081864548825024
max value change 0.26182590267308115
max value change 0.22241924489827625
max value change 0.18893482007007378
max value change 0.16048749469337054
max value change 0.1363216680780397
max value change 0.11579389463395273
max value change 0.09835690678710307
max value change 0.0835455294124472
max value change 0.0709644935583924
max value change 0.06027798807463114
max value change 0.051200741492550605
max value change 0.04349042748924603
max value change 0.03694120523971378
max value change 0.03137822849726035
max value change 0.02665297972288272
max value change 0.0226393059882

max value change 0.009557579923125559
max value change 0.008122496585826866
max value change 0.006902892595519461
max value change 0.005866413667490633
max value change 0.004985563424554584
max value change 0.00423697405119583
max value change 0.0036007864130169764
max value change 0.0030601232400044864
max value change 0.002600641404058024
max value change 0.0022101514159089675
max value change 0.0018782940504138423
max value change 0.0015962655391490443
max value change 0.0013565840073965774
max value change 0.0011528909972753354
max value change 0.0009797827812576543
max value change 0.0008326670085807564
max value change 0.0007076408779767007
max value change 0.0006013875980670491
max value change 0.0005110883981274128
max value change 0.00043434775159312267
max value change 0.000369129821478964
max value change 0.00031370445543643655
max value change 0.0002666012871941348
max value change 0.00022657072639731268
max value change 0.00019255081144819997
max value change 0.00016363903

policy stable False
Policy
  20    4    3    2    1    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  19    4    3    2    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  18    4    3    2    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 

  17    3    3    2    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   -1 

  16    3    2    2    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   -1 

  15    3    2    1    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   -1 

  14    3    2    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   -1 

  13    3    2    1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   -1 

  12    2    2    1    0    0    0    0    0    0    0    0    0    0    0   

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;代码6:确定环境状态动作值迭代程序代码实现：

In [ ]:
# 过程----确定环境状态动作值迭代
from mdpModel import MdpModel, Tools
import numpy as np
class StateActionValueIteration(MdpModel, Tools):
    def stateActionValueEval(self, sheet, q, theta, gamma, policy, printFlag, outputMidFlag):         # 阈值，设置为一个较小的实值
        self.q = q
        l = 0                                          # 迭代轮次
        while True:
            delta = 0                                    # 相对应状态值函数差值的最小值的绝对值，初始化为0
            qBackup = self.q.copy()
            for s in self.stateSpace():
                row, col = s
                actionNumber = 0
                for actionPi in policy[row * env.width + col]:
                    if actionPi > 0:
                        act = self.actionSpace()
                        a = act[actionNumber]
                        env.state = s
                        s1, _, _, _ = env.step(actionNumber, self.actionSpace())
                        r = self.rewardFunction(s, a)
                        p = self.transferFunction(s, a, s1)
                        actionNumber2 = 0
                        row1, col1 = s1
                        maxQValue = max(self.q[row1 * env.width + col1])
                        if maxQValue < -9999:
                            maxQValue = 0
                        self.q[row * env.width + col][actionNumber] = p * (r + gamma * maxQValue)
                        delta = max(delta, abs(self.q[row * env.width + col][actionNumber] - qBackup[row * env.width + col][actionNumber]))
                    else:
                        self.q[row * env.width + col][actionNumber] = -99999
                    actionNumber += 1
            l += 1        
            if printFlag:
                print('第%d轮迭代状态动作值'%l)
                self.printActionValue(self.q)
            if outputMidFlag:
                self.actionValuePolicyOutputExcel(sheet, self.q, policy, True, l)
            if delta < theta:
                break
        return self.q, l

    def optimalPolicy(self, qValue):
        stateActionSpace = self.stateActionSpacePi()
        optimPolicy = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
        for s in self.stateSpace():
            row, col = s
            actionNum = 0
            qMax = -9999
            for action in stateActionSpace[row * env.width + col]:
                qMax = max(qMax, qValue[row * env.width + col][actionNum])
                actionNum += 1
            actNum = 0
            actCount = list(stateActionSpace[row * env.width + col]).count(1)
            for action in stateActionSpace[row * env.width + col]:
                if action > 0:
                    if round(qValue[row * env.width + col][actNum], 5) == round(qMax, 5):
                        optimPolicy[row * env.width + col][actNum] = 1
                actNum += 1
            actCount = list(optimPolicy[row * env.width + col]).count(1)
            actNum = 0
            for act in list(optimPolicy[row * env.width + col]):
                if act == 1:
                    optimPolicy[row * env.width + col][actNum] = 1. /actCount
                actNum += 1
        return optimPolicy

In [ ]:
# 结果----确定环境状态动作值迭代结果
if __name__ == '__main__':
    stateActionValueIter = StateActionValueIteration(env)
    policy  = stateActionValueIter.stateActionSpacePi()
    q = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
    theta = 0.0001
    gamma = 0.8
    printFlag = True
    theta = 0.0001
    gamma = 0.8
    printFlag = True                                                  #是否打印评估过程
    outputMidFlag = True                                              #是否输出中间过程到Excel表
    if printFlag:
        print("迭代过程：")
    actionValueExcelFile = xlsxwriter.Workbook(r'.\resultTable\表4.6 确定环境机器人寻宝任务的动作值函数的值迭代过程表.xlsx')
    sheet = actionValueExcelFile.add_worksheet('sheet1')
    sheet.set_column('B:Z',35)
    q, l = stateActionValueIter.stateActionValueEval(sheet, q, theta, gamma, policy, printFlag, outputMidFlag)
    optimPolicy = stateActionValueIter.optimalPolicy(q)
    #print(optimPolicy)
    stateActionValueIter.actionValuePolicyOutputExcel(sheet, q, optimPolicy, False, l)
    actionValueExcelFile.close()
    print('\n\n确定环境机器人寻宝任务的状态动作值：')
    print("不可达状态：", end = '  ')
    for unreachstate in env.unreachState:
        print(unreachstate, end = '  ')
    print("\n终止状态：", end = '  ')
    for endstate in env.endState:
        print(endstate, end = '  ')
    print("\n特殊状态：", end = '  ')
    for typestate in env.typeState:
        print(typestate, end = '  ')
    print("\n是否允许冲出边界的动作", end = '  ')
    if edgeFlag:
        print("允许\n")
    else:
        print("不允许\n")
    stateActionValueIter.printActionValue(q)

迭代过程：
第1轮迭代状态动作值
状态       上       下       左       右
( 0, 0)    *.***    0.000    *.***    0.000
( 0, 1)    *.***    0.000    0.000    0.000
( 0, 2)    *.***    0.000    0.000    0.000
( 0, 3)    *.***    0.000    0.000    0.000
( 0, 4)    *.***    3.000    0.000    *.***
( 1, 0)    0.000    0.000    *.***    0.000
( 1, 1)    0.000    0.000    0.000    0.000
( 1, 2)    0.000  -10.000    0.000    0.000
( 1, 3)    0.000    0.000    0.000    3.000
( 1, 4)    *.***    *.***    *.***    *.***
( 2, 0)    0.000    0.000    *.***    0.000
( 2, 1)    0.000    0.000    0.000  -10.000
( 2, 2)    *.***    *.***    *.***    *.***
( 2, 3)    2.400    0.000   -8.080    0.000
( 2, 4)    3.000    0.000    1.920    *.***
( 3, 0)    0.000    1.000    *.***    0.000
( 3, 1)    0.000    0.000    0.800    0.000
( 3, 2)  -10.000    0.000    0.640    0.000
( 3, 3)    1.920    0.000    0.512    0.000
( 3, 4)    2.400    0.000    1.536    *.***
( 4, 0)    *.***    *.***    *.***    *.***
( 4, 1)    0.640    *.**

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;动画演示：

In [ ]:
import sys
import os
from time import sleep
import signal
import numpy as np
import random

def CtrlCHandler(signum, frame):
    env.render(close=True)    

def rendering(renderFlag, optimPolicy):
    if renderFlag:
        episodeNum = 10
        for i_episode in range(episodeNum):
            observation = env.reset()
            for t in range(100):
                signal.signal(signal.SIGINT, CtrlCHandler)
                env.render()
                sleep(0.3)
                row, col = observation
                actionlist = optimPolicy[row * env.width + col]
                a = -1
                while a <= 0:
                    aIndex = random.randrange(len(actionlist))
                    a = actionlist[aIndex]
                observation, reward, done, info = env.step(aIndex, actionAllList[:actionCount])
                if done:
                    env.render()
                    print("Episode finished after {} timesteps".format(t+1))
                    break
            sleep(2)
        env.render(close=True)
if __name__ =='__main__':
    rendering(True, optimPolicy)  

Episode finished after 3 timesteps
Episode finished after 1 timesteps
Episode finished after 2 timesteps
Episode finished after 1 timesteps
Episode finished after 5 timesteps
Episode finished after 1 timesteps
Episode finished after 1 timesteps
Episode finished after 5 timesteps
Episode finished after 3 timesteps
Episode finished after 1 timesteps


&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;代码7:确定环境状态值迭代程序代码实现：

In [ ]:
import numpy as np
from mdpModel import MdpModel, Tools


class StateValueIteration(MdpModel, Tools):

    def stateValueEval(self, sheet, v, theta, gamma, printFlag, outputMidFlag):  # 阈值，设置为一个较小的实值
        self.v = v
        l = 0  # 迭代轮次
        while True:
            delta = 0  # 相对应状态值函数差值的最小值的绝对值，初始化为0
            vBackup = self.v.copy()
            for s in self.stateSpace():
                if not self.env.isEndState(s) and not self.env.isNoReachState(s):
                    row, col = s
                    maxValue = -2
                    actionNum = 0
                    stateActionSpace = self.stateActionSpacePi()
                    for action in stateActionSpace[row * env.width + col]:
                        if action > 0:
                            act = self.actionSpace()
                            a = act[actionNum]
                            env.state = s
                            s1, _, _, _ = env.step(actionNum, act)
                            r = self.rewardFunction(s, a)
                            row1, col1 = s1
                            maxValue = max(maxValue, r + gamma * self.v[row1][col1])
                        actionNum += 1
                    self.v[row][col] = maxValue
                    delta = max(delta, abs(self.v[row][col] - vBackup[row][col]))
            l += 1
            if printFlag:
                print('第%d轮迭代状态值' % l)
                self.printStateValue(self.v)
                print('——————————————————————————————————————')
            if outputMidFlag:
                self.stateValueIterOutputExcel(sheet, v, q, initPolicy, False, l)
            if delta < theta:
                break
        return self.v, l

    def optimalPolicy(self, v):
        optimPolicy = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
        qValue = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
        for s in self.stateSpace():
            row, col = s
            qMax = -9999
            actionNum = 0
            stateActionSpace = self.stateActionSpacePi()
            for action in stateActionSpace[row * env.width + col]:
                stateValue = 0
                if action > 0:
                    act = self.actionSpace()
                    a = act[actionNum]
                    env.state = s
                    s1, _, _, _ = env.step(actionNum, act)
                    p = self.transferFunction(s, a, s1)
                    r = self.rewardFunction(s, a)
                    row1, col1 = s1
                    qValue[row * env.width + col][actionNum] = qValue[row * env.width + col][actionNum] + p * (r + gamma * self.v[row1][col1])
                else:
                    qValue[row * env.width + col][actionNum] = -99999
                qMax = max(qMax, qValue[row * env.width + col][actionNum])
                actionNum += 1
            actNum = 0
            actCount = list(stateActionSpace[row * env.width + col]).count(1)
            for action in stateActionSpace[row * env.width + col]:
                if action > 0:
                    if round(qValue[row * env.width + col][actNum], 5) == round(qMax, 5):
                        optimPolicy[row * env.width + col][actNum] = 1
                actNum += 1
            actCount = list(optimPolicy[row * env.width + col]).count(1)
            actNum = 0
            for act in list(optimPolicy[row * env.width + col]):
                if act == 1:
                    optimPolicy[row * env.width + col][actNum] = 1. / actCount
                actNum += 1
        return optimPolicy, qValue

In [ ]:
if __name__ == '__main__':
    stateValueIter = StateValueIteration(env)
    stateValueIterExcelFile = xlsxwriter.Workbook(r'.\resultTable\表4.5 面向确定环境机器人寻宝任务的状态值迭代更新过程表.xlsx')
    sheet = stateValueIterExcelFile.add_worksheet('sheet1')
    sheet.set_column('B:Z',35)
    print("值迭代过程：")
    theta = 0.0001
    gamma = 0.8
    printFlag = True
    outputMidFlag = True
    l = 0
    v = np.array(([[0.0 for col in range(env.width)] for row in range(env.height)]))
    q = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
    initPolicy = np.array(([[0.0 for col in range(env.actionCount)] for row in range(env.height * env.width)]))
    stateValueIter.stateValueIterOutputExcel(sheet, v, q, initPolicy, False, l)
    v, l = stateValueIter.stateValueEval(sheet, v, theta, gamma, printFlag, outputMidFlag)
    optimalPolicy, q = stateValueIter.optimalPolicy(v)
    print("最优动作值：")
    stateValueIter.printActionValue(q)
    print("最优策略：")
    stateValueIter.printActionValue(optimalPolicy)
    stateValueIter.stateValueIterOutputExcel(sheet, v, q, optimalPolicy, True, l)
    stateValueIterExcelFile.close()

值迭代过程：
第1轮迭代状态值
    0.000     0.000     0.000     0.000     3.000 
    0.000     0.000     0.000     3.000     0.000 
    0.000     0.000     0.000     2.400     3.000 
    1.000     0.800     0.640     1.920     2.400 
    0.000     1.000     0.800     1.536     1.920 
——————————————————————————————————————
第2轮迭代状态值
    0.000     0.000     0.000     2.400     3.000 
    0.000     0.000     2.400     3.000     0.000 
    0.800     0.640     0.000     2.400     3.000 
    1.000     0.800     1.536     1.920     2.400 
    0.000     1.000     1.229     1.536     1.920 
——————————————————————————————————————
第3轮迭代状态值
    0.000     0.000     1.920     2.400     3.000 
    0.640     1.920     2.400     3.000     0.000 
    0.800     1.536     0.000     2.400     3.000 
    1.000     1.229     1.536     1.920     2.400 
    0.000     1.000     1.229     1.536     1.920 
——————————————————————————————————————
第4轮迭代状态值
    0.512     1.536     1.920     2.400     3.000 
    1.536     1.920     

&emsp;&emsp;&emsp;&emsp;<font color=#ff0000>$\blacktriangleright$&emsp;代码8:赌徒问题值迭代评估程序代码实现：

In [ ]:


import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# goal
GOAL = 100

# all states, including state 0 and state 100
STATES = np.arange(GOAL + 1)

# probability of head
HEAD_PROB = 0.4

def figure_4_3():
    # state value
    state_value = np.zeros(GOAL + 1)
    state_value[GOAL] = 1.0

    # value iteration
    iter = 1
    while True:
        delta = 0.0
        print('iter:',iter)
        for state in STATES[1:GOAL]:
            # get possilbe actions for current state
            actions = np.arange(1,min(state, GOAL - state) + 1)
            action_returns = []
            for action in actions:
                action_returns.append(
                    (HEAD_PROB * state_value[state + action] + (1 - HEAD_PROB) * state_value[state - action]))
            print('state=',state,'actions=',actions)
            print('action_returns=',np.round(action_returns,3))
            new_value = np.max(action_returns)
            delta += np.abs(state_value[state] - new_value)
            # update state value
            state_value[state] = new_value
            print('state_value:',np.round(state_value,3))
        iter += 1
        if delta < 1e-9:
            break

    # compute the optimal policy
    policy = np.zeros(GOAL + 1)
    for state in STATES[1:GOAL]:
        actions = np.arange(min(state, GOAL - state) + 1)
        action_returns = []
        for action in actions:
            action_returns.append(
                HEAD_PROB * state_value[state + action] + (1 - HEAD_PROB) * state_value[state - action])

        maxNum = max(action_returns)
        print('state:',state)
        print('action value:',np.round(action_returns,3))
        for action in actions:
            if np.abs(maxNum - action_returns[action])<0.0001:
                print('action',action)
        
        # round to resemble the figure in the book, see
        # https://github.com/ShangtongZhang/reinforcement-learning-an-introduction/issues/83
        policy[state] = actions[np.argmax(np.round(action_returns[1:], 5)) + 1]
        print('###',state,'***',policy[state])
        
    maxNum = max(policy)
    
    plt.figure(figsize=(10, 20))

    plt.subplot(2, 1, 1)
    plt.plot(state_value)
    plt.xlabel('Capital')
    plt.ylabel('Value estimates')

    plt.subplot(2, 1, 2)
    plt.scatter(STATES, policy)
    plt.xlabel('Capital')
    plt.ylabel('Final policy (stake)')

    plt.savefig('images/figure_4_3.png')
    plt.close()

if __name__ == '__main__':
    figure_4_3()


iter: 1
state= 1 actions= [1]
action_returns= [0.]
state_value: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
state= 2 actions= [1 2]
action_returns= [0. 0.]
state_value: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
state= 3 actions= [1 2 3]
action_returns= [0. 0. 0.]
state_value: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

state= 22 actions= [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
action_returns= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
state_value: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
state= 23 actions= [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
action_returns= [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
state_value: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1.]
state= 2

state_value: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.4 0.4 0.4
 0.4 0.4 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]
state= 56 actions= [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
action_returns= [0.24 0.24 0.24 0.24 0.24 0.24 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.4 ]
state_value: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0

state_value: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.4 0.4 0.4
 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. ]
state= 70 actions= [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30]
action_returns= [0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24 0.24
 0.24 0.24 0.24 0.24 0.24 0.24 0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.4 ]
state_value: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.4 0.4 0.4 0.4
 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0

state_value: [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.4  0.4  0.4  0.4  0.4  0.4
 0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4  0.4
 0.4  0.4  0.4  0.4  0.4  0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64 0.64
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   1.  ]
state= 84 actions= [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
action_returns= [0.384 0.384 0.384 0.384 0.384 0.384 0.384 0.384 0.384 0.24  0.24  0.24
 0.24  0.24  0.24  0.64 ]
state_value: [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.4  0.4  0.4  0.4  0.4  0.4
 0.4  0.4 

...

state_value: [0.    0.002 0.005 0.009 0.013 0.017 0.023 0.028 0.032 0.038 0.043 0.05
 0.058 0.065 0.07  0.074 0.081 0.087 0.094 0.103 0.109 0.116 0.126 0.134
 0.144 0.16  0.163 0.168 0.174 0.179 0.186 0.195 0.202 0.208 0.217 0.225
 0.236 0.246 0.258 0.264 0.272 0.281 0.29  0.301 0.315 0.323 0.334 0.349
 0.36  0.376 0.4   0.403 0.408 0.414 0.419 0.426 0.435 0.442 0.448 0.457
 0.465 0.476 0.486 0.498 0.504 0.512 0.521 0.53  0.541 0.555 0.563 0.574
 0.589 0.6   0.616 0.64  0.645 0.652 0.661 0.669 0.679 0.692 0.703 0.713
 0.725 0.738 0.753 0.77  0.787 0.796 0.807 0.822 0.835 0.852 0.872 0.884
 0.901 0.923 0.941 0.964 1.   ]
state= 2 actions= [1 2]
action_returns= [0.005 0.005]
state_value: [0.    0.002 0.005 0.009 0.013 0.017 0.023 0.028 0.032 0.038 0.043 0.05
 0.058 0.065 0.07  0.074 0.081 0.087 0.094 0.103 0.109 0.116 0.126 0.134
 0.144 0.16  0.163 0.168 0.174 0.179 0.186 0.195 0.202 0.208 0.217 0.225
 0.236 0.246 0.258 0.264 0.272 0.281 0.29  0.301 0.315 0.323 0.334 0.349
 0.36  0.376 0

state= 36 actions= [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36]
action_returns= [0.234 0.233 0.231 0.23  0.229 0.228 0.228 0.23  0.23  0.231 0.236 0.231
 0.231 0.236 0.231 0.228 0.227 0.224 0.222 0.222 0.221 0.221 0.222 0.221
 0.22  0.221 0.222 0.221 0.221 0.222 0.222 0.224 0.227 0.228 0.231 0.236]
state_value: [0.    0.002 0.005 0.009 0.013 0.017 0.023 0.028 0.032 0.038 0.043 0.05
 0.058 0.065 0.07  0.074 0.081 0.087 0.094 0.103 0.109 0.116 0.126 0.134
 0.144 0.16  0.163 0.168 0.174 0.179 0.186 0.195 0.202 0.208 0.217 0.225
 0.236 0.246 0.258 0.264 0.272 0.281 0.29  0.301 0.315 0.323 0.334 0.349
 0.36  0.376 0.4   0.403 0.408 0.414 0.419 0.426 0.435 0.442 0.448 0.457
 0.465 0.476 0.486 0.498 0.504 0.512 0.521 0.53  0.541 0.555 0.563 0.574
 0.589 0.6   0.616 0.64  0.645 0.652 0.661 0.669 0.679 0.692 0.703 0.713
 0.725 0.738 0.753 0.77  0.787 0.796 0.807 0.822 0.835 0.852 0.872 0.884
 0.901 0.923 0.941 0.964 1.   ]
state=

state_value: [0.    0.002 0.005 0.009 0.013 0.017 0.023 0.028 0.032 0.038 0.043 0.05
 0.058 0.065 0.07  0.074 0.081 0.087 0.094 0.103 0.109 0.116 0.126 0.134
 0.144 0.16  0.163 0.168 0.174 0.179 0.186 0.195 0.202 0.208 0.217 0.225
 0.236 0.246 0.258 0.264 0.272 0.281 0.29  0.301 0.315 0.323 0.334 0.349
 0.36  0.376 0.4   0.403 0.408 0.414 0.419 0.426 0.435 0.442 0.448 0.457
 0.465 0.476 0.486 0.498 0.504 0.512 0.521 0.53  0.541 0.555 0.563 0.574
 0.589 0.6   0.616 0.64  0.645 0.652 0.661 0.669 0.679 0.692 0.703 0.713
 0.725 0.738 0.753 0.77  0.787 0.796 0.807 0.822 0.835 0.852 0.872 0.884
 0.901 0.923 0.941 0.964 1.   ]
state= 57 actions= [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43]
action_returns= [0.44  0.438 0.438 0.439 0.439 0.441 0.442 0.43  0.425 0.421 0.417 0.416
 0.414 0.41  0.409 0.409 0.409 0.415 0.413 0.409 0.406 0.403 0.402 0.402
 0.402 0.402 0.402 0.403 0.406 0.409 0.413 0.415 0.409 0.40

state_value: [0.    0.002 0.005 0.009 0.013 0.017 0.023 0.028 0.032 0.038 0.043 0.05
 0.058 0.065 0.07  0.074 0.081 0.087 0.094 0.103 0.109 0.116 0.126 0.134
 0.144 0.16  0.163 0.168 0.174 0.179 0.186 0.195 0.202 0.208 0.217 0.225
 0.236 0.246 0.258 0.264 0.272 0.281 0.29  0.301 0.315 0.323 0.334 0.349
 0.36  0.376 0.4   0.403 0.408 0.414 0.419 0.426 0.435 0.442 0.448 0.457
 0.465 0.476 0.486 0.498 0.504 0.512 0.521 0.53  0.541 0.555 0.563 0.574
 0.589 0.6   0.616 0.64  0.645 0.652 0.661 0.669 0.679 0.692 0.703 0.713
 0.725 0.738 0.753 0.77  0.787 0.796 0.807 0.822 0.835 0.852 0.872 0.884
 0.901 0.923 0.941 0.964 1.   ]
state= 88 actions= [ 1  2  3  4  5  6  7  8  9 10 11 12]
action_returns= [0.78  0.775 0.771 0.769 0.768 0.77  0.769 0.768 0.771 0.773 0.777 0.787]
state_value: [0.    0.002 0.005 0.009 0.013 0.017 0.023 0.028 0.032 0.038 0.043 0.05
 0.058 0.065 0.07  0.074 0.081 0.087 0.094 0.103 0.109 0.116 0.126 0.134
 0.144 0.16  0.163 0.168 0.174 0.179 0.186 0.195 0.202 0.208 0.217 

action 36
### 64 *** 11.0
state: 65
action value: [0.512 0.511 0.511 0.508 0.507 0.504 0.503 0.505 0.505 0.507 0.512 0.509
 0.509 0.509 0.509 0.512 0.502 0.497 0.494 0.49  0.489 0.49  0.489 0.489
 0.487 0.486 0.487 0.489 0.489 0.49  0.489 0.49  0.494 0.497 0.502 0.512]
action 0
action 10
action 15
action 35
### 65 *** 10.0
state: 66
action value: [0.521 0.519 0.519 0.521 0.517 0.515 0.515 0.514 0.516 0.521 0.519 0.516
 0.516 0.516 0.516 0.519 0.521 0.511 0.506 0.504 0.502 0.502 0.504 0.499
 0.497 0.497 0.497 0.499 0.504 0.502 0.502 0.504 0.506 0.511 0.521]
action 0
action 9
action 16
action 34
### 66 *** 9.0
state: 67
action value: [0.53  0.529 0.529 0.528 0.528 0.527 0.525 0.526 0.53  0.527 0.526 0.525
 0.523 0.523 0.525 0.526 0.527 0.53  0.521 0.518 0.517 0.515 0.512 0.512
 0.509 0.508 0.509 0.512 0.512 0.515 0.517 0.518 0.521 0.53 ]
action 0
action 8
action 17
action 33
### 67 *** 8.0
state: 68
action value: [0.541 0.54  0.538 0.537 0.538 0.539 0.538 0.541 0.537 0.535 0.533 0.533
 0

## 3. 实验题

### 测试一：编程序完成下列任务

&emsp;&emsp;（编程）分别利用基于动作值函数的策略迭代和值迭代算法求解4x4格子世界的最优策略：如图所示4x4格子世界，共有16个状态，其中两个阴影位置为终止状态。在每个状态下，都可以执行4个动作：上、下、左、右。奖赏设置：除了到达终止状态获得0的奖赏以外，其他情况均得到-1的奖赏。agent试图离开该区域，得到-1奖赏，并保持原地不动。
<center><img src='./image/GridWorld2.png' width='350'></center>
